In [2]:
import pandas as pd
import numpy as np

# Simulamos un reporte de ventas con varios problemas
data_ventas = {
    'cliente': ['Ana', 'Luis', np.nan, 'Pedro', 'Sofia'],      # Fila 2: Falta Cliente (Crítico)
    'ventas': [100.0, np.nan, 300.0, 450.0, 200.0],            # Fila 1: Falta Monto
    'fecha': ['01/01/2024', '02/01/2024', np.nan, '04/01/2024', '05/01/2024'], # Fila 2: Falta Fecha
    'zona': ['Norte', 'Sur', 'Norte', np.nan, 'Oeste']         # Fila 3: Falta Zona
}

data_set = pd.DataFrame(data_ventas)
print("---  ESTADO INICIAL ---")
print(data_set)

---  ESTADO INICIAL ---
  cliente  ventas       fecha   zona
0     Ana   100.0  01/01/2024  Norte
1    Luis     NaN  02/01/2024    Sur
2     NaN   300.0         NaN  Norte
3   Pedro   450.0  04/01/2024    NaN
4   Sofia   200.0  05/01/2024  Oeste


# Ejemplo 1: La Opción Nuclear (dropna)

Contexto: Tenemos una fila donde no sabemos quién es el cliente (NaN). Una venta sin cliente no sirve para facturar. Debemos eliminarla.

In [3]:
# Eliminamos la fila completa, PERO solo si falta el dato en 'cliente'
df_limpio = data_set.dropna(subset=['cliente'])
print("--- Filas eliminadas (Adiós al cliente fantasma)----")
print(df_limpio)

--- Filas eliminadas (Adiós al cliente fantasma)----
  cliente  ventas       fecha   zona
0     Ana   100.0  01/01/2024  Norte
1    Luis     NaN  02/01/2024    Sur
3   Pedro   450.0  04/01/2024    NaN
4   Sofia   200.0  05/01/2024  Oeste


Explicación:

    .dropna(subset=['cliente']): Le ordena a Pandas eliminar la fila completa, pero únicamente si el valor nulo se encuentra en la columna cliente. Si hubiera un nulo en ventas o zona, la fila sobreviviría. Es una eliminación quirúrgica.

# Ejemplo 2: Relleno Simple (fillna)

Contexto: Ahora vemos que a Luis le falta el monto de ventas. No queremos borrar a Luis, preferimos asumir que si está vacío, es porque vendió 0.

In [4]:
# Rellenamos los huecos de la columna ventas con el número 0
df_limpio['ventas'] = df_limpio['ventas'].fillna(0)

print("---- Ventas corregidas (Luis ahora tiene 0)----")
print(df_limpio['ventas'])

---- Ventas corregidas (Luis ahora tiene 0)----
0    100.0
1      0.0
3    450.0
4    200.0
Name: ventas, dtype: float64


Explicación:

    df['ventas'] = ...: Estamos sobreescribiendo la columna original con la versión corregida.

    .fillna(0): Busca cualquier casilla vacía (NaN) en esa columna específica y le incrusta el valor 0. Los datos que ya existían (como 100.0 o 450.0) se quedan intactos.

# Ejemplo 3: Relleno hacia adelante (ffill)

Contexto: Falta una fecha. En registros diarios, es común asumir que si no se escribió la fecha, pertenece al mismo día del registro anterior o sigue la secuencia. Usaremos la lógica de "arrastrar el valor anterior".

In [5]:
# Rellenamos la fecha usando el método 'forward fill' (relleno hacia adelante)
df_limpio['fecha'] = df_limpio['fecha'].ffill()

print("--- Fecha Completadas----")
print(df_limpio['fecha'])


--- Fecha Completadas----
0    01/01/2024
1    02/01/2024
3    04/01/2024
4    05/01/2024
Name: fecha, dtype: str


Explicación:

    .ffill(): Significa Forward Fill. Pandas mira el valor válido anterior (en este caso '02/01/2024') y lo copia en la celda vacía siguiente. Es ideal para series de tiempo o datos ordenados cronológicamente.

# Ejemplo 4: El Detective (Bandera + Relleno)

Contexto: A Pedro le falta la zona. El jefe dice: "Ponle 'Desconocido', pero quiero que guardes un registro de que ese dato fue manipulado por nosotros".

In [6]:
# Paso 1: Crear la evidencia (Bandera Booleana)
df_limpio['zona_era_nula'] = df_limpio['zona'].isna()

# Paso 2: Arreglar el dato(Imputación)
df_limpio['zona'] = df_limpio['zona'].fillna('Desconocido')

print("--- Zona arreglada con auditoría ---")
print(df_limpio[['cliente', 'zona', 'zona_era_nula']])


--- Zona arreglada con auditoría ---
  cliente         zona  zona_era_nula
0     Ana        Norte          False
1    Luis          Sur          False
3   Pedro  Desconocido           True
4   Sofia        Oeste          False


Explicación:

    .isna(): Escanea la columna y devuelve True si encuentra un hueco y False si hay un dato. Al guardar esto en zona_era_nula, creamos una "pista de auditoría".

    .fillna('Desconocido'): Ahora que ya guardamos la evidencia, procedemos a tapar el hueco con la palabra 'Desconocido' para que el reporte quede limpio visualmente.